In [ ]:
# This is only to fix autocomplete in my version of Jupyter notebook.  Disregard if you don't have this problem
%config Completer.use_jedi = False

# This is to allow the use pyqtgraph for plots
%gui qt

# Imports and config

In [ ]:
from easy_wave import Block_Writer, AWG_Writer, Channel, to_integer, PulseLengthError
from wave_library import *
from visual_wave import plot_line, plot_block
from tqdm import tqdm_notebook

# First let's give names to our channels
chs = {
    'sg1_i' : Channel.ch1_a,
    'sg1_g' : Channel.ch1_m1,
    'offres': Channel.ch1_m2,
    
    'sg1_q'  : Channel.ch2_a,
    'lockin' : Channel.ch2_m1,
    'readout': Channel.ch2_m2,
    
    'pico_trig': Channel.ch3_a,
    'red'      : Channel.ch3_m1,
    'rocket'   : Channel.ch3_m2,
    
    'res_analog': Channel.ch4_a,
    'solar'     : Channel.ch4_m1,
    'hyper'     : Channel.ch4_m2,
}
sg1_chs = [chs[x] for x in ['sg1_i','sg1_q', 'sg1_g']]

shifts = {
            chs['solar']: -470e-9,
            chs['offres']: -390e-9,
        }

working_dir = r"2019-08 Isotopically Purified/"

# Some common pulses sets

In [ ]:
def basic_pulses(mw_pi = 100e-9, square_Ahpi=0.8888, init_time = 200e-6, readout_time = 100e-6,
                 delay_time = 2e-6, init_laser = 'offres', read_laser = 'hyper', mw_buffer=50e-9):
    """These are a basic set of init, read and square mw pulses"""
    init = Rect(t=init_time, ch=chs[init_laser])
    readout  = Rect(t=readout_time, ch=chs[read_laser])
    readout &= Rect(t=readout_time, ch=chs['readout'])
    delay = Zero(t=delay_time, ch=Channel.no_ch)
    wait = lambda _t: Zero(t=_t, ch=Channel.no_ch)
    mw = lambda t, phase, A: IQ_MW_Pulse(t, *sg1_chs, phase=phase, A=A, freq_shift=0, mw_buffer=mw_buffer)
    pi = lambda phase: mw(mw_pi, phase, 1)
    hpi = lambda phase: mw(mw_pi/2, phase, square_Ahpi)
    return init, readout, delay, wait, mw, pi, hpi  

In [ ]:
def basic_compilled_pulses(rate, init_laser='offres', read_laser='hyper', init_time = 200e-6, readout_time = 100e-6, **kwargs):
    """These are a basic set of init, read and square mw pulses"""
    init, readout, delay, wait, mw, pi, hpi = basic_pulses(**kwargs)
    get_div = lambda name, time: to_integer(time*rate/250, PulseLengthError("{} time is not divisible by 250/rate\n\tpulse_time: {:.0f}ns \n\t250/rate: {:.0f}ns".format(name ,time*1e9, 250/rate*1e9)))
    
    init_div = get_div('init', init_time)
    init = Rect(t=init_time/init_div, ch=chs[init_laser])
  
    read_div = get_div('readout', readout_time)
    def readout(hi_cycle):
        line  = Rect(t=readout_time/read_div, ch=chs[read_laser])
        line &= Rect(t=readout_time/read_div, ch=chs['readout'])
        if hi_cycle:
            line &= Rect(line.t, ch=chs['lockin'])
        return line
    
    return init, readout, delay, wait, mw, pi, hpi, init_div, read_div

# Basic Waveforms and Calibrations

## CW ODMR
(untested)

In [ ]:
def gen(writer=AWG_Writer(shifts=shifts)):
    cycle_time = 1e-6
    line = Rect(t=cycle_time, ch=chs['readout'])
    line &= Rect(t=cycle_time, ch=chs['offres'])
    for ch_name in ['sg1_i', 'sg1_g', 'lockin']:
        line &= RectCycle(t=cycle_time, ch=chs[ch_name])
    writer.add_line(line, 'odmr')
    return writer
writer = gen().generate_and_upload('192.168.1.101', working_dir + 'cw_odmr.awg', rate=1e9, limits={})
plot_line(writer, 1)

## Square Rabi

In [ ]:
def gen(writer=AWG_Writer(shifts=shifts)):
    taus = np.arange(0, 200e-9, 2e-9)
    
    init, readout, delay, wait, _mw, pi, hpi = basic_pulses()
    mw = lambda tau: _mw(tau, 0, 1)
    buffer = lambda tau: wait(max(taus)-tau)
    for tau in tqdm_notebook(taus):
        line  = init + delay + mw(tau)         + delay + readout + buffer(tau) + delay + delay
        line += init + delay + mw(tau).blank() + delay + readout + buffer(tau) + delay + delay
        line &= RectCycle(t=line.t, ch=chs['lockin'])
        writer.add_line(line, 'resonant sq rabi {:.0f}ns'.format(tau / 1e-9))
    return writer
writer = gen().generate_and_upload('192.168.1.101', working_dir + 'square_rabi.awg', rate=5e8, limits={})
plt = plot_line(writer, 10)

## Repeated Pi Calibration

### Repeated Pi

In [ ]:
def gen(writer=AWG_Writer(shifts=shifts)):
    pi_time = 100e-9
    init, readout, delay, wait, mw, pi, hpi = basic_pulses(mw_pi=pi_time)
    N, factor = 10, 10
    for i in tqdm_notebook(range(N)):
        number_of_pi = i*factor + 1
        seq = (pi(0) + wait(pi_time))*number_of_pi
        buffer = (pi(0).blank() + wait(pi_time))*(N*factor-number_of_pi)
        line  = delay + init + delay + seq         + buffer + readout
        line += delay + init + delay + seq.blank() + buffer + readout
        line &= RectCycle(t=line.t, ch=chs['lockin'])
        writer.add_line(line, 'Test i={} (pi={:.0f}ns)'.format(number_of_pi, pi_time/ 1e-9))
    return writer
writer = gen().generate_and_upload('192.168.1.101', working_dir + 'repeated pi cal.awg', rate=1e8, limits={})
plt = plot_line(writer, 10)

### Repeated Half-Pi
(untested)

In [ ]:
def gen(writer=AWG_Writer(shifts=shifts)):
    pi_time = 100e-9
    init, readout, delay, wait, mw, pi, hpi = basic_pulses(mw_pi=pi_time, square_Ahpi=0.8888)
    N, factor = 10, 10
    for i in tqdm_notebook(range(N)):
        number_of_hpi = 2*(i*factor + 1)
        seq = (hpi(0) + wait(pi_time))*number_of_hpi
        buffer = (pi(0).blank() + wait(pi_time))*(2*N*factor-number_of_pi)
        line  = delay + init + delay + seq         + buffer + readout
        line += delay + init + delay + seq.blank() + buffer + readout
        line &= RectCycle(t=line.t, ch=chs['lockin'])
        writer.add_line(line, 'Test i={} (pi={:.0f}ns)'.format(number_of_pi, pi_time/ 1e-9))
    return writer
writer = gen().generate_and_upload('192.168.1.101', working_dir + 'repeated pi cal.awg', rate=1e8, limits={})
plt = plot_line(writer, 10)

## Ramsey (T2*) / Hahn (T2)

In [ ]:
def gen(writer=AWG_Writer(shifts=shifts), seq_type='hahn'):
    init, readout, delay, wait, mw, pi, hpi = basic_pulses()
    if seq_type == 'ramsey':
        wait_seq = lambda t: wait(t)
        taus = np.arange(0,200e-6, 1e-6)
    elif seq_type == 'hahn':
        wait_seq = lambda t: wait(t/2) + pi(90) + wait(t/2)
        taus = np.arange(0,4000e-6, 40e-6)
    
    for t in tqdm_notebook(taus):
        line  = delay + init + delay + hpi(0) + wait_seq(t) + hpi(0)   + delay + readout + wait(max(taus)-t)
        line += delay + init + delay + hpi(0) + wait_seq(t) + hpi(180) + delay + readout + wait(max(taus)-t)
        line &= RectCycle(t=line.t, ch=chs['lockin'])
        writer.add_line(line, 'Coherence t={:.3f}us'.format(t*1e6))
    writer.generate_and_upload('192.168.1.101', working_dir + '{}.awg'.format(seq_type), rate=2e7, limits={})
    return writer
writer = gen(seq_type='hahn')
# plot_line(writer, 10)

## PLE

### PLE with Off-Res Init

### CW-PLE with MW
(untested)

In [ ]:
res_laser='hyper'
def gen(writer=AWG_Writer(shifts=shifts)):
    cycle_time = 1e-6
    line = Rect(t=cycle_time, ch=chs[res_laser])
    for ch_name in ['sg1_i', 'sg1_g', 'readout']:
        line &= Rect(t=cycle_time, ch=chs[ch_name])
    writer.add_line(line, 'ple')
    return writer
writer = gen().generate_and_upload('192.168.1.101', working_dir + 'cw_mw_ple_{}.awg'.format(res_laser), rate=1e9, limits={})
plot_line(writer, 1)

# Weakly Coupled

## XY8-N (compilled)

In [ ]:
def gen(writer=Block_Writer(shifts=shifts), rate=1e8):
    init, readout, delay, wait, mw, pi, hpi, init_div, read_div= basic_compilled_pulses(rate)
    
    N = 16
    taus = np.arange(200e-9, 6e-6, 10e-9)
    
    #Define the XY8 sequence
    def xy8(tau, loc='middle'):
        x, y = pi(0), pi(90)
        w = wait(tau-(x.t/2))
        hw = wait(tau-x.t)
        base = x+w + w+y+w + w+x+w + w+y+w + w+y+w + w+x+w + w+y+w + w+x
        if loc == 'middle':  return w + base + w
        elif loc == 'begin': return hw + base + w
        elif loc == 'end':   return w + base + hw
        else:  raise Exception("Unrecongnize loc!")
    
    #Make the blocks
    for tau in tqdm_notebook(taus):
        writer.new_block('xy8-N tau={:.0f}ns'.format(tau*1e9))
        lo, hi = ' (lo)', ' (hi)'
        
        #Lo cycle
        writer.add_line(init,                               'init'+lo, repeat=init_div)
        writer.add_line(hpi(90) + xy8(tau,'begin'),         'hpi_begin'+lo, repeat=1)
        writer.add_line(xy8(tau,'middle'),                  'xy8'+lo, repeat=N-2)
        writer.add_line(xy8(tau,'end') + hpi(90),           'hpi_end'+lo, repeat=1)
        writer.add_line(readout(False),                     'read'+lo, repeat=read_div)
        writer.add_line(wait((250/rate) + 16*(max(taus)-tau)), 'buffer'+lo, repeat=N)
        
        #Hi cycle
        writer.add_line(init,                               'init'+hi, repeat=init_div)
        writer.add_line(hpi(90) + xy8(tau,'begin'),         'hpi_begin'+hi, repeat=1)
        writer.add_line(xy8(tau,'middle'),                  'xy8'+hi, repeat=N-2)
        writer.add_line(xy8(tau,'end') + hpi(-90),          'hpi_end'+hi, repeat=1)
        writer.add_line(readout(True),                      'read'+hi, repeat=read_div)
        writer.add_line(wait((250/rate) + 16*(max(taus)-tau)), 'buffer'+hi, repeat=N)

#     writer.generate_and_upload('192.168.1.101', working_dir + 'xy8-N.awg', rate=rate, limits={})
    return writer
writer = gen()
writer.print_info()
plot_block(writer, int(6960/12)-1)

In [ ]:
from easy_wave import AWG_Writer, REAL_CHANNELS
from wave_library import Empty
import pyqtgraph as pg
import time
def plot_wave(wave, plot_zeros=True, downsample=10, downsample_mode='peak'):
    win = pg.GraphicsWindow()
    cs = lambda ch: ['y','b','#FF00FF','g'][ch.value[0]-1]
    plt = win.addPlot()
    plt.hideAxis('left')

    # Optimize for large data
    plt.setClipToView(True)
    if downsample:
        plt.setDownsampling(ds=downsample, auto=True, mode=downsample_mode)

    start = time.time()
    dy = 0
    offset=3
    for i,ch in enumerate(REAL_CHANNELS):
        if ch in wave.chs or plot_zeros:
            ts, ys = wave.generate(1e9, ch)
            print(ys.shape)
            ys = ys.astype(float) + dy
            plt.plot(ts, ys, pen=cs(ch), fillLevel = dy, brush=(50,50,200,100))
            dy -= offset
            if plot_zeros and (i+1)%3==0:
                dy-=1
    print(time.time()-start)
    
    #Set to zooming limit
    plt.vb.setLimits(yMax=1.5, yMin=dy, xMin=0, xMax=wave.t)
                
    # Add crosshair
    label = pg.TextItem("t=0.000 us", anchor=(0,0))

    vLine = pg.InfiniteLine(angle=90, movable=False)
    plt.addItem(vLine)
    label.setParentItem(plt)
    def mouseMoved(pos):
        if plt.sceneBoundingRect().contains(pos):
            mousePoint = plt.vb.mapSceneToView(pos)
            if 0<mousePoint.x()<wave.t:
                label.setText("t={:.3f} us".format(mousePoint.x()*1e6))
                vLine.setPos(mousePoint.x())
#     proxy = pg.SignalProxy(plt_item.scene().sigMouseMoved, rateLimit=60, slot=mouseMoved)
    plt.scene().sigMouseMoved.connect(mouseMoved)
    
    return win

def plot_line(writer, line, plot_zeros=True, downsample=10, downsample_mode='peak'):
    wave = writer.lines[line-1]['waveform']
    return plot_wave(wave, plot_zeros=plot_zeros, downsample=downsample, downsample_mode=downsample_mode)

def plot_block(writer, block, plot_zeros=True, downsample=10, downsample_mode='peak'):
    if type(block) == int:
        block_name = list(writer.blocks.keys())[block]
    elif type(block) == str:
        block_name = block
    else:
        raise Exception("Unrecongnize type for <block> to be plotted")
    b = writer.blocks[block_name]
    wave = Empty()
    for lname, line in b.items():
        wave += line['waveform']*line['repeat']
        
    return plot_wave(wave, plot_zeros=plot_zeros, downsample=downsample, downsample_mode=downsample_mode)

In [ ]:
%%time
plot_block(writer, 10)